In [ ]:
#import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, to_graphviz
from sklearn.metrics import classification_report, plot_confusion_matrix,roc_curve, accuracy_score, roc_auc_score, auc
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import matplotlib.pyplot as plt
import graphviz
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn")
import seaborn as sns
import time

In [ ]:
seeds = pd.read_csv("/content/drive/MyDrive/seeds.csv")
seeds.drop("ID",axis=1,inplace=True)
colnames = np.array(seeds.drop("seedType",axis=1).columns)

In [ ]:
IQR = seeds.asymmetryCoefficient.quantile(0.75) - seeds.asymmetryCoefficient.quantile(0.25)
uppB = seeds.asymmetryCoefficient.quantile(0.75) + 1.5*IQR
seeds[seeds.asymmetryCoefficient >= uppB]
seedsNew = seeds.drop(index=[203,207],axis=0)
X = seedsNew.drop("seedType",axis=1)
y = seedsNew.seedType -1 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,stratify=y,test_size=0.25)

In [ ]:
param_grid = {"n_estimators":np.arange(1,500,50),
              "criterion":["gini","entropy"],
              "max_depth":np.arange(1,20),
              "min_samples_split":[1,2,3,4],
              "max_features":["auto","sqrt","log2"],
              "bootstrap":[True,False]}
clf = GridSearchCV(RandomForestClassifier(),param_grid,n_jobs=-1,verbose=1)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 9120 candidates, totalling 45600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 8446 tasks      | elapsed: 32.7min
[Parallel(n_jobs=-1)]: Done 9796 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 11246 tasks      |

Geçen süre 10115.14570236206


In [ ]:
model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=18, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=401,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False);
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=18, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=401,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88        18
           1       0.94      1.00      0.97        17
           2       0.85      1.00      0.92        17

    accuracy                           0.92        52
   macro avg       0.93      0.93      0.92        52
weighted avg       0.93      0.92      0.92        52



In [ ]:
roc_auc_score(y_test,model.predict_proba(X_test),multi_class="ovr")*100

98.56987239340181